In [ ]:
!ngrok authtoken ENTER INFO HERE

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install Flask
# !pip install flask-ngrok
# !pip install flask-ngrok pyngrok

In [ ]:
!npm install -g localtunnel

In [ ]:
import subprocess
import time
import requests
import re

# Function to start localtunnel and get the URL
def start_localtunnel():
    process = subprocess.Popen(['lt', '--port', '5000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for _ in range(10):  # Try for a certain number of lines
        output = process.stdout.readline().decode('utf-8')
        url_match = re.search(r'http[s]?://[^\s]+', output)
        if url_match:
            return url_match.group(0)
        time.sleep(1)  # Wait a bit before reading the next line

    process.terminate()
    raise Exception("Failed to get the public URL from localtunnel.")

# Start localtunnel and get the URL
public_url = start_localtunnel()
print(f'Public URL: {public_url}')


In [ ]:
from flask import Flask, request, jsonify
# from flask_ngrok import run_with_ngrok
# from pyngrok import ngrok
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import ast
import re

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

dir="/content/drive/MyDrive/BridgeAthletics/Training/AdvancedData2"
original_model_name = "t5-base"
model_name = dir+'/final_model_'+original_model_name
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

if device.type=="cuda":
  model.half().to(device)
model.eval()

def str_to_list(generated):
    s = generated
    s = s.replace("</s>", "").strip()
    s = s.strip("<pad>")
    s = s.strip(" ### Response: ")
    try:
        s = ast.literal_eval(s)
    except (ValueError, SyntaxError):  #POST PROCESSING
      last_bracket_index = s.rfind('}')
      if last_bracket_index != -1:
          s = s[:last_bracket_index + 1]
          s += ']'
          try:
              s=ast.literal_eval(s)
          except (ValueError, SyntaxError):
              return -1
    if type(s) != list or not all(isinstance(item, dict) for item in s):
      return -1
    else:
      return s

def check_parameters_correctness(model_output_list, param_list=['reps','time','distance']):
    required_keys = {'exercise', 'sets'}
    for dictionary in (model_output_list):
        if not required_keys.issubset(dictionary.keys()):
            return -2

        allowed_keys = required_keys.union(param_list)
        if not set(dictionary.keys()).issubset(allowed_keys):
          return -3
    return 1

def get_set_and_param_consistency(model_output_list,param_list=['reps','time','distance']):
  required_keys = {'exercise', 'sets'}
  for d in range(len(model_output_list)):
    sets = model_output_list[d]['sets']
    if sets == 0:
      return -1
    for param in param_list:
      if param in model_output_list[d]:
        param_len = len(model_output_list[d][param])
        if sets > param_len:
            last_value = model_output_list[d][param][-1]
            model_output_list[d][param].extend([last_value] * (sets - param_len))
        elif sets < param_len:
            model_output_list[d][param] = model_output_list[d][param][:sets]
  return model_output_list

def remove_consecutive_duplicates(model_outputs_list):
    cleaned_output = []
    for j in range(len(model_outputs_list)):
      if j == 0 or model_outputs_list[j]['exercise'] != model_outputs_list[j-1]['exercise']:
        cleaned_output.append(model_outputs_list[j])
    return cleaned_output


def format_model_input(text):
  def normalize_userinput(text):
    text = text.lower()
    text = re.sub(r'[:/\\.]', ',', text)
    text = re.sub(r'\-', '', text)
    text = re.sub(r'\+', 'and', text)
    text = re.sub(r'\&', 'and', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s*,\s*', ', ', text)
    if ((text.startswith("'") and text.endswith("'")) or (text.startswith('"') and text.endswith('"'))):
      text=text[1:-1]
    return text
  instruction = f"### Instruction:\nGenerate a workout block in JSON format for the following block title."
  normalized_input = normalize_userinput(text)
  input_text = f"\n\n### Input:\n{normalized_input}"
  return instruction + input_text

app = Flask(__name__)
# run_with_ngrok(app)
@app.route('/predict', methods=['POST'])
def predict():
    print("Running")
    try:
        data = request.get_json(force=True)
        input_text = data['input_text']
        input_text = format_model_input(input_text)
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

        with torch.no_grad():
            output = model.generate(
                input_ids=input_ids,
                eos_token_id=1,
                max_length=400,
                num_beams=2,
                num_return_sequences=1,
                early_stopping=True,
                repetition_penalty=3.0,
                do_sample=True,
                top_k=9,
                top_p=0.93,
                temperature=1.2,
            )

        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
        stop_sequence = "### Response"
        stop_index = decoded_output.find(stop_sequence, decoded_output.find(stop_sequence) + len(stop_sequence))
        if stop_index != -1:
            output = decoded_output[:stop_index]
        else:
            output = decoded_output

        output_list = str_to_list(output)
        if output_list == -1:
            return jsonify({'output_text': f"Error in output {output}"})

        if check_parameters_correctness(output_list) != 1:
            return jsonify({'output_text': f"Error in output {output}"})

        output_list = get_set_and_param_consistency(output_list)
        if output_list == -1:
            return jsonify({'output_text': f"Error in output {output}"})

        output_list = remove_consecutive_duplicates(output_list)

        return jsonify({'output_text': output_list})
    except Exception as e:
        return jsonify({'error': str(e)})


if __name__ == '__main__':
  # public_url = ngrok.connect(5000)
  # print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))
  app.run(port=5000)


In [ ]:
from flask import Flask, request, jsonify
# from flask_ngrok import run_with_ngrok
# from pyngrok import ngrok
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import ast
import re

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

dir="/content/drive/MyDrive/BridgeAthletics/Training/AdvancedData2"
original_model_name = "t5-base"
model_name = dir+'/final_model_'+original_model_name+"AWS"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

if device.type=="cuda":
  model.half().to(device)
model.eval()

def str_to_list(generated):
    s = generated
    s = s.replace("</s>", "").strip()
    s = s.strip("<pad>")
    s = s.strip(" ### Response: ")
    try:
        s = ast.literal_eval(s)
    except (ValueError, SyntaxError):  #POST PROCESSING
      last_bracket_index = s.rfind('}')
      if last_bracket_index != -1:
          s = s[:last_bracket_index + 1]
          s += ']'
          try:
              s=ast.literal_eval(s)
          except (ValueError, SyntaxError):
              return -1
    if type(s) != list or not all(isinstance(item, dict) for item in s):
      return -1
    else:
      return s

def check_parameters_correctness(model_output_list, param_list=['reps','time','distance']):
    required_keys = {'exercise', 'sets'}
    for dictionary in (model_output_list):
        if not required_keys.issubset(dictionary.keys()):
            return -2

        allowed_keys = required_keys.union(param_list)
        if not set(dictionary.keys()).issubset(allowed_keys):
          return -3
    return 1

def get_set_and_param_consistency(model_output_list,param_list=['reps','time','distance']):
  required_keys = {'exercise', 'sets'}
  for d in range(len(model_output_list)):
    sets = model_output_list[d]['sets']
    if sets == 0:
      return -1
    for param in param_list:
      if param in model_output_list[d]:
        param_len = len(model_output_list[d][param])
        if sets > param_len:
            last_value = model_output_list[d][param][-1]
            model_output_list[d][param].extend([last_value] * (sets - param_len))
        elif sets < param_len:
            model_output_list[d][param] = model_output_list[d][param][:sets]
  return model_output_list

def remove_consecutive_duplicates(model_outputs_list):
    cleaned_output = []
    for j in range(len(model_outputs_list)):
      if j == 0 or model_outputs_list[j]['exercise'] != model_outputs_list[j-1]['exercise']:
        cleaned_output.append(model_outputs_list[j])
    return cleaned_output


def format_model_input(text):
  def normalize_userinput(text):
    text = text.lower()
    text = re.sub(r'[:/\\.]', ',', text)
    text = re.sub(r'\-', '', text)
    text = re.sub(r'\+', 'and', text)
    text = re.sub(r'\&', 'and', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s*,\s*', ', ', text)
    if ((text.startswith("'") and text.endswith("'")) or (text.startswith('"') and text.endswith('"'))):
      text=text[1:-1]
    return text
  instruction = f"### Instruction:\nGenerate a workout block in JSON format for the following block title."
  normalized_input = normalize_userinput(text)
  input_text = f"\n\n### Input:\n{normalized_input}"
  return instruction + input_text

In [ ]:
def predict(input_text):
  input_text = format_model_input(input_text)
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

  with torch.no_grad():
      output = model.generate(
          input_ids=input_ids,
          eos_token_id=1,
          max_length=400,
          num_beams=2,
          num_return_sequences=1,
          early_stopping=True,
          repetition_penalty=3.0,
          do_sample=True,
          top_k=9,
          top_p=0.96,
          temperature=1.2,
      )

  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  stop_sequence = "### Response"
  stop_index = decoded_output.find(stop_sequence, decoded_output.find(stop_sequence) + len(stop_sequence))
  if stop_index != -1:
      output = decoded_output[:stop_index]
  else:
      output = decoded_output

  output_list = str_to_list(output)
  if output_list == -1:
      return f"Error in output {output}"

  if check_parameters_correctness(output_list) != 1:
      return f"Error in output {output}"

  output_list = get_set_and_param_consistency(output_list)
  if output_list == -1:
      return f"Error in output {output}"

  output_list = remove_consecutive_duplicates(output_list)

  return output_list

In [ ]:
predict("arms")